In [1]:
from xgboost import XGBRegressor
import pandas as pd
from sklearn.metrics import mean_squared_error
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [3]:
train_df.head(5)

,Unnamed: 0,user_id,business_id,rating,rounded_business_rating,latitude,longitude,average_user_rating,num_user_reviews,avg_user_compliments,...,name_le,max_user_likes,total_user_likes,avg_user_likes,max_business_likes,total_business_likes,avg_business_likes,max_ub_pair_likes,total_ub_pair_likes,avg_ub_pair_likes
0,0,pmKnNkGVgzhMkwJaTU5apA,yfxDa8RFOvJPQh0rNtakHA,4.0,3.5,36.126194,-115.193445,3.74,959,0.045881,...,94743,0.0,0.0,0.000000,1.0,7.0,0.006416,0.0,0.0,0.0
1,1,cQU8IWVEJYcklBJbW9CzdQ,yfxDa8RFOvJPQh0rNtakHA,4.0,3.5,36.126194,-115.193445,3.59,289,0.179931,...,94743,0.0,0.0,0.000000,1.0,7.0,0.006416,0.0,0.0,0.0
2,2,GmQR50bIII7gDNRYEgfXfA,yfxDa8RFOvJPQh0rNtakHA,5.0,3.5,36.126194,-115.193445,4.07,1261,0.298176,...,94743,0.0,0.0,0.000000,1.0,7.0,0.006416,0.0,0.0,0.0
3,3,m4VAw2u8gHrANfXP7CW2mg,yfxDa8RFOvJPQh0rNtakHA,4.0,3.5,36.126194,-115.193445,4.03,58,0.103448,...,94743,1.0,5.0,0.046729,1.0,7.0,0.006416,0.0,0.0,0.0
4,4,mvK1pv-x0Z5rP4Hz9cWBWw,yfxDa8RFOvJPQh0rNtakHA,4.0,3.5,36.126194,-115.193445,3.88,1641,6.855576,...,94743,2.0,2.0,0.181818,1.0,7.0,0.006416,0.0,0.0,0.0


In [4]:
test_df_with_labels = pd.read_csv('y_test.csv')
test_df_with_labels

,user_id,business_id,rating
0,m5b84-nPPVNLlDtZyVGZ3w,tWdp26XQEJqJGnRmFVJQxA,4.0
1,m5b84-nPPVNLlDtZyVGZ3w,qy4WKXTBkqD4zX0rohYF8Q,4.0
2,m5b84-nPPVNLlDtZyVGZ3w,478TIlfHXfT3wvww54QsPg,3.0
3,m5b84-nPPVNLlDtZyVGZ3w,SxvKjtbq5qRlFoXrU8Bo9Q,2.0
4,m5b84-nPPVNLlDtZyVGZ3w,ErTB7R4rJNUUV8nsUVG26Q,4.0
...,...,...,...
142039,iCqpb2C3TZrB5gdMASf8LQ,N8D-5-fW7EkBdKjNYGhu0A,5.0
142040,iCqpb2C3TZrB5gdMASf8LQ,dT9316C_MJjATvv7GByROg,4.0
142041,iCqpb2C3TZrB5gdMASf8LQ,SAIrNOB4PtDA4gziNCucwg,4.0
142042,iCqpb2C3TZrB5gdMASf8LQ,K7lWdNUhCbcnEvI0NhGewg,4.0


In [5]:
final_test_df = pd.merge(test_df, test_df_with_labels, on=['user_id', 'business_id'])
final_test_df

,Unnamed: 0,user_id,business_id,rounded_business_rating,latitude,longitude,average_user_rating,num_user_reviews,avg_user_compliments,reviews_per_year,...,max_user_likes,total_user_likes,avg_user_likes,max_business_likes,total_business_likes,avg_business_likes,max_ub_pair_likes,total_ub_pair_likes,avg_ub_pair_likes,rating
0,0,NCRGj36ihdwX-VUCYEX41g,LNGBEEelQx4zbfWnlc66cw,4.0,35.965529,-115.168686,4.13,317,0.378549,42.742889,...,0.0,0.0,0.000000,2.0,11.0,0.012373,0.0,0.0,0.0,2.0
1,1,mySDXsBhY3u0yI-G2oVDkA,LNGBEEelQx4zbfWnlc66cw,4.0,35.965529,-115.168686,2.73,239,0.200837,23.469196,...,0.0,0.0,0.000000,2.0,11.0,0.012373,0.0,0.0,0.0,2.0
2,2,wLc6l8v7O9hXjrDi3Q7tyw,LNGBEEelQx4zbfWnlc66cw,4.0,35.965529,-115.168686,3.96,102,0.666667,11.978764,...,1.0,1.0,0.004808,2.0,11.0,0.012373,0.0,0.0,0.0,5.0
3,3,hnhs-uzv_eoE6Pq8nXyufA,LNGBEEelQx4zbfWnlc66cw,4.0,35.965529,-115.168686,3.84,203,0.394089,25.142518,...,0.0,0.0,0.000000,2.0,11.0,0.012373,0.0,0.0,0.0,5.0
4,4,Gr-MqCunME2K_KmsAwjpTA,LNGBEEelQx4zbfWnlc66cw,4.0,35.965529,-115.168686,4.12,336,1.309524,37.922078,...,1.0,2.0,0.080000,2.0,11.0,0.012373,0.0,0.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142039,142039,NTuvVb-ZwQ_rFn6W9Krm7A,a9WV4-QcQfkUcOwUMFVhGw,4.5,36.076360,-115.081695,3.92,430,0.218605,54.973730,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,5.0
142040,142040,qPganFH_fTXrP9r3kXf6YQ,aapwKXeDH_hsjthI13DI0w,4.0,45.572295,-73.754008,3.53,145,0.013793,28.968254,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,4.0
142041,142041,zbRjeEkON8EXFk_VrcTAiA,QSMWY0fXT19xLfSVe9nXJg,1.5,36.078242,-115.093153,3.29,137,0.043796,13.588315,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,1.0
142042,142042,FTsRfaVbDUB6Sc6pL-J80w,oZJHJieidHz-zuGL7lBMpg,4.5,36.239382,-115.203632,3.26,53,0.000000,8.291899,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,1.0


In [6]:
final_test_df.keys()

Index(['Unnamed: 0', 'user_id', 'business_id', 'rounded_business_rating',
       'latitude', 'longitude', 'average_user_rating', 'num_user_reviews',
       'avg_user_compliments', 'reviews_per_year', 'city', 'state', 'name',
       'attributes', 'review_count', 'city_state', 'user_business_pair',
       'price_range', 'bike_parking', 'business_accepts_credit_cards',
       'good_for_kids', 'has_TV', 'outdoor_seating', 'restaurants_delivery',
       'restaurants_good_for_groups', 'restaurants_reservations',
       'restaurants_take_out', 'restaurants_counter_service',
       'by_appointment_only', 'wheelchair_accessible', 'open_24_hours',
       'dogs_allowed', 'caters', 'drive_thru', 'coat_check',
       'good_for_dancing', 'restaurants_table_service', 'happy_hour',
       'business_accepts_bitcoin', 'garage', 'street', 'validated', 'lot',
       'valet', 'lunch', 'dinner', 'casual', 'city_state_le', 'name_le',
       'max_user_likes', 'total_user_likes', 'avg_user_likes',
       'max_

In [8]:
X_train = train_df[['rounded_business_rating', 'latitude', 'longitude', 'average_user_rating', 'num_user_reviews', 'avg_user_compliments', \
    'reviews_per_year', 'review_count', 'price_range', 'bike_parking', 'good_for_kids', 'outdoor_seating', 'restaurants_delivery', \
        'restaurants_take_out', 'by_appointment_only', 'wheelchair_accessible', 'dogs_allowed', 'caters', 'drive_thru', 'coat_check', \
            'restaurants_table_service', 'business_accepts_bitcoin', 'garage', 'street', 'lot', 'valet', 'lunch', 'dinner', 'casual', \
                'city_state_le', 'name_le', 'max_user_likes', 'total_user_likes', 'avg_user_likes', 'max_business_likes', \
                    'total_business_likes', 'avg_business_likes', 'avg_ub_pair_likes']]
y_train = train_df['rating']

xgb = XGBRegressor(objective='reg:linear', base_score=0.5, booster='gbtree', colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1, \
    gamma=0, gpu_id=1, interaction_constraints='', learning_rate=0.300000012, max_delta_step=0, max_depth=6, min_child_weight=1, \
        n_jobs=8, num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1, \
            tree_method='exact', validate_parameters=1, verbosity=None).fit(X_train, y_train)

y_pred_train = list(xgb.predict(X_train))
train_rmse = mean_squared_error(list(y_train), y_pred_train, squared=False)

X_test = final_test_df[['rounded_business_rating', 'latitude', 'longitude', 'average_user_rating', 'num_user_reviews', 'avg_user_compliments', \
    'reviews_per_year', 'review_count', 'price_range', 'bike_parking', 'good_for_kids', 'outdoor_seating', 'restaurants_delivery', \
        'restaurants_take_out', 'by_appointment_only', 'wheelchair_accessible', 'dogs_allowed', 'caters', 'drive_thru', 'coat_check', \
            'restaurants_table_service', 'business_accepts_bitcoin', 'garage', 'street', 'lot', 'valet', 'lunch', 'dinner', 'casual', \
                'city_state_le', 'name_le', 'max_user_likes', 'total_user_likes', 'avg_user_likes', 'max_business_likes', \
                    'total_business_likes', 'avg_business_likes', 'avg_ub_pair_likes']]
y_test = final_test_df['rating']
y_pred_test = list(xgb.predict(X_test))
test_rmse = mean_squared_error(list(y_test), y_pred_test, squared=False)

print(f'Train RMSE: {train_rmse}; Test RMSE: {test_rmse}')

Train RMSE: 0.9630116041852984; Test RMSE: 0.9798764280445167


In [9]:
space={'learning_rate': hp.uniform("learning_rate", 0.01, 0.5),
    'max_depth': hp.quniform("max_depth", 3, 18, 1),
    'gamma': hp.uniform ('gamma', 0, 9),
    'reg_alpha': hp.quniform('reg_alpha', 0, 100, 1),
    'reg_lambda': hp.uniform('reg_lambda', 0, 100),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
    'min_child_weight': hp.quniform('min_child_weight', 0, 10, 1),
    'n_estimators': hp.quniform("n_estimators", 100, 1000, 1),
    'subsample': hp.uniform("subsample", 0.5, 1),
    'random_state': 0
}

def objective(space):
    clf = XGBRegressor(objective='reg:linear', base_score=0.5, booster='gbtree', colsample_bylevel=1, colsample_bynode=1, \
        gpu_id=1, interaction_constraints='', max_delta_step=0, \
            n_jobs=8, num_parallel_tree=1, random_state=0, scale_pos_weight=1, subsample=space['subsample'], \
                tree_method='exact', validate_parameters=1, verbosity=None,
                    n_estimators=int(space['n_estimators']), max_depth=int(space['max_depth']), gamma=space['gamma'],
                        reg_alpha=int(space['reg_alpha']), min_child_weight=int(space['min_child_weight']), reg_lambda=space['reg_lambda'],
                            colsample_bytree=space['colsample_bytree'], learning_rate=space['learning_rate'])
    
    evaluation = [( X_train, y_train), ( X_test, y_test)]
    
    clf.fit(X_train, y_train,
            eval_set=evaluation, eval_metric="rmse",
            early_stopping_rounds=10,verbose=False)
    

    pred = clf.predict(X_test)
    test_rmse = mean_squared_error(y_test, pred, squared=False)
    print ("Test RMSE:", test_rmse)
    return {'loss': test_rmse, 'status': STATUS_OK }

In [10]:
trials = Trials()

best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 25,
                        trials = trials)

Test RMSE:                                            
0.9810786253456674                                    
Test RMSE:                                                                      
0.9788805453822071                                                              
Test RMSE:                                                                      
0.9803092378124538                                                              
Test RMSE:                                                                       
0.9812425161408153                                                               
Test RMSE:                                                                       
0.9807293087442007                                                               
Test RMSE:                                                                       
0.9794868355102607                                                              
Test RMSE:                                                                 

KeyboardInterrupt: 

In [17]:
best_hyperparams

{'colsample_bytree': 0.7670554549316846,
 'gamma': 4.886927896270191,
 'learning_rate': 0.04262522175688959,
 'max_depth': 17.0,
 'min_child_weight': 8.0,
 'n_estimators': 574.0,
 'reg_alpha': 63.0,
 'reg_lambda': 34.99906842816348}

In [20]:
xgb_best = XGBRegressor(objective='reg:linear', base_score=0.5, booster='gbtree', colsample_bylevel=1, colsample_bynode=1, \
        gpu_id=1, interaction_constraints='', max_delta_step=0, random_state=0, \
            n_jobs=8, num_parallel_tree=1, scale_pos_weight=1, subsample=best_hyperparams['subsample'], \
                tree_method='exact', validate_parameters=1, verbosity=None, n_estimators=int(best_hyperparams['n_estimators']), \
                    max_depth=int(best_hyperparams['max_depth']), reg_lambda=best_hyperparams['reg_lambda'], \
                        gamma=best_hyperparams['gamma'], learning_rate=best_hyperparams['learning_rate'], \
                            reg_alpha=int(best_hyperparams['reg_alpha']), min_child_weight=int(best_hyperparams['min_child_weight']), \
                                colsample_bytree=best_hyperparams['colsample_bytree']).fit(X_train, y_train)

y_pred_train = list(xgb_best.predict(X_train))
train_rmse = mean_squared_error(list(y_train), y_pred_train, squared=False)

y_pred_test = list(xgb_best.predict(X_test))
test_rmse = mean_squared_error(list(y_test), y_pred_test, squared=False)

print(f'Train RMSE: {train_rmse}; Test RMSE: {test_rmse}')

Train RMSE: 0.9684958237327738; Test RMSE: 0.9781164498955753


In [152]:
from sklearn.model_selection import RandomizedSearchCV, KFold
from scipy import stats

space={'learning_rate': stats.uniform(0.01, 0.49),
    'max_depth': stats.randint(1, 10),
    'gamma': stats.uniform (1, 8),
    'reg_alpha': stats.uniform(0, 100),
    'reg_lambda': stats.uniform(0, 100),
    'colsample_bytree': stats.uniform(0.5, 0.5),
    'min_child_weight': stats.randint(0, 10),
    'n_estimators': stats.randint(100, 990)
}

xgb_test = XGBRegressor()
cv = KFold(n_splits=5, shuffle=True)
rscv = RandomizedSearchCV(xgb_test, space, n_iter=100, cv=cv)
rscv.fit(X_train, y_train)
best_model = rscv.best_estimator_
best_model

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7108151246534398,
             gamma=2.2447654903089527, gpu_id=-1, importance_type='gain',
             interaction_constraints='', learning_rate=0.08765832581870968,
             max_delta_step=0, max_depth=9, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=777, n_jobs=8,
             num_parallel_tree=1, random_state=0, reg_alpha=34.28128842482173,
             reg_lambda=30.63049228235448, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [153]:
xgb_best_rscv = XGBRegressor(n_estimators=int(best_hyperparams['n_estimators']), max_depth=int(best_hyperparams['max_depth']),
                    reg_lambda=int(best_hyperparams['reg_lambda']), gamma=best_hyperparams['gamma'],
                    learning_rate=best_hyperparams['learning_rate'], reg_alpha=int(best_hyperparams['reg_alpha']),
                    min_child_weight=int(best_hyperparams['min_child_weight']), 
                    colsample_bytree=int(best_hyperparams['colsample_bytree'])).fit(X_train, y_train)

y_pred_train = list(xgb_best_rscv.predict(X_train))
train_rmse = mean_squared_error(list(y_train), y_pred_train, squared=False)

y_pred_test = list(xgb_best_rscv.predict(X_test))
test_rmse = mean_squared_error(list(y_test), y_pred_test, squared=False)

print(f'Train RMSE: {train_rmse}; Test RMSE: {test_rmse}')

Train RMSE: 0.9758865449213205; Test RMSE: 0.9831840967979896


In [79]:
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [80]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [82]:
X_train_scaled

array([[0.75      , 0.11663196, 0.70949721, 0.00142725, 0.01611441,
        0.00522837],
       [0.625     , 0.11272616, 0.70949721, 0.00142725, 0.01611441,
        0.00522837],
       [0.875     , 0.11265137, 0.70949721, 0.00142725, 0.01611441,
        0.00522837],
       ...,
       [0.375     , 0.81993327, 0.60055866, 0.04777097, 0.00432245,
        0.03629749],
       [0.625     , 0.81934843, 0.60055866, 0.04777097, 0.00432245,
        0.03629749],
       [0.625     , 0.820775  , 0.60055866, 0.04777097, 0.00432245,
        0.03629749]])

In [83]:
X_test_scaled

array([[0.75      , 0.11594999, 0.65083799, 0.02846109, 0.00245943,
        0.02096952],
       [0.5       , 0.11486258, 0.65083799, 0.02846109, 0.00245943,
        0.02096952],
       [0.625     , 0.11059656, 0.65083799, 0.02846109, 0.00245943,
        0.02096952],
       ...,
       [0.75      , 0.1123162 , 0.79050279, 0.00167912, 0.00267644,
        0.00206003],
       [0.5       , 0.11242984, 0.79050279, 0.00167912, 0.00267644,
        0.00206003],
       [0.625     , 0.10241453, 0.79050279, 0.00167912, 0.00267644,
        0.00206003]])

In [84]:
mlp = MLPRegressor().fit(X_train_scaled, y_train)
y_pred_train = list(mlp.predict(X_train_scaled))
train_rmse = mean_squared_error(list(y_train), y_pred_train, squared=False)

y_pred_test = list(mlp.predict(X_test_scaled))
test_rmse = mean_squared_error(list(y_test), y_pred_test, squared=False)

print(f'Train RMSE: {train_rmse}; Test RMSE: {test_rmse}')

Train RMSE: 0.9873522112644619; Test RMSE: 0.9849081866497436


In [13]:
space={'solver': 'adam',
    'learning_rate_init': hp.loguniform("max_depth", -14, -2),
    'learning_rate': 'invscaling',
    'alpha': hp.loguniform('alpha', -14, 5),
    'early_stopping': True,
    'max_iter': hp.quniform('max_iter', 25, 200, 1),
    'batch_size': hp.quniform("batch_size", 50, 256, 1),
    'seed': 0
}

def objective_mlp(space):
    clf = MLPRegressor(
                    solver=space['solver'], learning_rate_init=space['learning_rate_init'], learning_rate=space['learning_rate'],
                    alpha=space['alpha'], early_stopping=space['early_stopping'], max_iter=int(space['max_iter']),
                    batch_size=int(space['batch_size']))
        
    clf.fit(X_train_scaled, y_train)
    

    pred = clf.predict(X_test_scaled)
    test_rmse = mean_squared_error(y_test, pred, squared=False)
    print ("RMSE:", test_rmse)
    return {'loss': test_rmse, 'status': STATUS_OK }

In [14]:
trials = Trials()

best_hyperparams = fmin(fn = objective_mlp,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 100,
                        trials = trials)

RMSE:                                                  
0.9882866812848835                                     
RMSE:                                                                              
0.9876771389463073                                                                 
  2%|▏         | 2/100 [01:35<1:17:48, 47.64s/trial, best loss: 0.9876771389463073]

/Users/rpenumarti/opt/anaconda3/envs/dsci553/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (48) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)



RMSE:                                                                              
1.1206573594362608                                                                 
RMSE:                                                                              
1.0290160299626325                                                               
RMSE:                                                                              
1.0085652332775241                                                                 
RMSE:                                                                              
1.0430502448412622                                                                 
RMSE:                                                                              
0.9932047909478007                                                                 
RMSE:                                                                              
0.9924073195082076                                                            

/Users/rpenumarti/opt/anaconda3/envs/dsci553/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (26) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)



RMSE:                                                                              
1.0136458763783958                                                                 
RMSE:                                                                              
0.9928038341904493                                                               
RMSE:                                                                             
0.988147444829406                                                                 
RMSE:                                                                             
1.017178200141302                                                                 
RMSE:                                                                             
0.9970298072586634                                                                
 13%|█▎        | 13/100 [11:41<1:54:56, 79.27s/trial, best loss: 0.9876771389463073]

...

RMSE:                                                                               
0.993175017352215                                                                   
RMSE:                                                                                
0.9872115761709152                                                                   
RMSE:                                                                                
0.9932608229459529                                                                   
RMSE:                                                                                
0.9892327403832943                                                                  
RMSE:                                                                               
0.9923904756642049                                                                  
RMSE:                                                                               
0.9871494341104109                                          

/Users/rpenumarti/opt/anaconda3/envs/dsci553/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (27) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)



RMSE:                                                                               
0.987649925555707                                                                   
RMSE:                                                                               
0.9969576655720611                                                                  
 97%|█████████▋| 97/100 [1:27:51<01:38, 32.80s/trial, best loss: 0.9869474577673258]

/Users/rpenumarti/opt/anaconda3/envs/dsci553/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (49) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)



RMSE:                                                                               
0.9884344779910732                                                                  
RMSE:                                                                               
0.9888258390275899                                                                  
RMSE:                                                                               
0.9878504481214172                                                                  
100%|██████████| 100/100 [1:29:38<00:00, 53.79s/trial, best loss: 0.9869474577673258]


In [15]:
best_hyperparams

{'alpha': 1.1229576726515301e-05,
 'batch_size': 73.0,
 'max_depth': 0.0012930826800508551,
 'max_iter': 77.0}

In [17]:
mlp_best = MLPRegressor(solver=space['solver'],
                    learning_rate=space['learning_rate'], alpha=best_hyperparams['alpha'], early_stopping=space['early_stopping'], 
                    max_iter=int(best_hyperparams['max_iter']), batch_size=int(best_hyperparams['batch_size'])).fit(X_train, y_train)
y_pred_train = list(mlp_best.predict(X_train))
train_rmse = mean_squared_error(list(y_train), y_pred_train, squared=False)

y_pred_test = list(mlp_best.predict(X_test))
test_rmse = mean_squared_error(list(y_test), y_pred_test, squared=False)

print(f'Train RMSE: {train_rmse}; Test RMSE: {test_rmse}')

Train RMSE: 0.9900167346453688; Test RMSE: 0.9874462773725613


In [18]:
from sklearn.model_selection import GridSearchCV, KFold

In [19]:
param_grid = {}
param_grid["hidden_layer_sizes"] = [(50,), (100,), (200,), (400,), (50, 25,), (100, 50,), (200, 100,), (400, 200,),
                                        (50, 25, 10,), (100, 50, 25,), (200, 100, 50,), (400, 200, 100,)]
param_grid["learning_rate_init"] = [1e-4, 1e-3, 1e-2, 1e-1]

cv = KFold(n_splits=5, shuffle=True)
gscv = GridSearchCV(mlp_best, param_grid, n_jobs=-1, cv=cv)
gscv.fit(X_train, y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=True),
             estimator=MLPRegressor(alpha=1.1229576726515301e-05, batch_size=73,
                                    early_stopping=True,
                                    learning_rate='invscaling', max_iter=77),
             n_jobs=-1,
             param_grid={'hidden_layer_sizes': [(50,), (100,), (200,), (400,),
                                                (50, 25), (100, 50), (200, 100),
                                                (400, 200), (50, 25, 10),
                                                (100, 50, 25), (200, 100, 50),
                                                (400, 200, 100)],
                         'learning_rate_init': [0.0001, 0.001, 0.01, 0.1]})

In [20]:
gscv.best_estimator_

MLPRegressor(alpha=1.1229576726515301e-05, batch_size=73, early_stopping=True,
             hidden_layer_sizes=(400,), learning_rate='invscaling',
             max_iter=77)

In [21]:
mlp_best = MLPRegressor(solver=space['solver'], learning_rate_init=gscv.best_estimator_.learning_rate_init,
                    learning_rate=space['learning_rate'], alpha=best_hyperparams['alpha'], early_stopping=space['early_stopping'], 
                    max_iter=int(best_hyperparams['max_iter']), batch_size=int(best_hyperparams['batch_size']),
                    hidden_layer_sizes=gscv.best_estimator_.hidden_layer_sizes).fit(X_train, y_train)
y_pred_train = list(mlp_best.predict(X_train))
train_rmse = mean_squared_error(list(y_train), y_pred_train, squared=False)

y_pred_test = list(mlp_best.predict(X_test))
test_rmse = mean_squared_error(list(y_test), y_pred_test, squared=False)

print(f'Train RMSE: {train_rmse}; Test RMSE: {test_rmse}')

Train RMSE: 0.989775429064536; Test RMSE: 0.9872883086808237


In [98]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch import optim
import numpy as np
from math import sqrt
from torch.utils.data import Dataset, DataLoader

In [86]:
dev = None
if torch.cuda.is_available():
  dev = torch.device('cuda:0')
else:
  dev = torch.device('cpu')

print(dev)

cpu


In [143]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(6, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 1)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [131]:
class Data(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X.astype(np.float32))
        self.y = torch.from_numpy(y.values)
        self.len = self.X.shape[0]
       
    def __getitem__(self, index):
        return self.X[index], self.y[index]
   
    def __len__(self):
        return self.len

In [132]:
batch_size = 64

train_data = Data(X_train_scaled, y_train)
train_dataloader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)

test_data = Data(X_test_scaled, y_test)
test_dataloader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=True)

In [133]:
for batch, (X, y) in enumerate(train_dataloader):
    print(f"Batch: {batch+1}")
    print(f"X shape: {X.shape}")
    print(f"y shape: {y.shape}")
    break

Batch: 1
X shape: torch.Size([64, 6])
y shape: torch.Size([64])


In [144]:
num_epochs = 25
loss_function = nn.MSELoss()
model = Net()
model.to(dev)
optimizer = optim.Adam(model.parameters(), lr=0.01)

test_pred = None
test_y = None

for epoch in range(num_epochs):
    total_loss = 0
    for i, (X, y) in enumerate(train_dataloader):
        optimizer.zero_grad()
       
        pred = model(X)
        loss = torch.sqrt(loss_function(pred, y.unsqueeze(-1).float()))
        loss.backward()
        optimizer.step()

        if epoch == num_epochs-1:
            if i == 0:
                test_pred = pred
                test_y = y.unsqueeze(-1).float()
            else:
                test_pred = torch.cat((test_pred, pred), 0)
                test_y = torch.cat((test_y, y.unsqueeze(-1).float()), 0)

        total_loss += loss.item()
    
    print(f'Epoch {epoch+1}: Loss - {total_loss}')

Epoch 1: Loss - 7300.769656896591
Epoch 2: Loss - 7212.214529037476
Epoch 3: Loss - 7205.682536363602
Epoch 4: Loss - 7200.915259361267
Epoch 5: Loss - 7171.814963042736
Epoch 6: Loss - 7154.1936329603195
Epoch 7: Loss - 7145.818837761879
Epoch 8: Loss - 7144.839111089706
Epoch 9: Loss - 7143.698472559452
Epoch 10: Loss - 7138.083162724972
Epoch 11: Loss - 7137.669512093067
Epoch 12: Loss - 7137.924142479897
Epoch 13: Loss - 7138.158192813396
Epoch 14: Loss - 7133.418288290501
Epoch 15: Loss - 7137.479692697525
Epoch 16: Loss - 7138.433169305325
Epoch 17: Loss - 7137.042835831642
Epoch 18: Loss - 7136.410615026951
Epoch 19: Loss - 7139.914788365364
Epoch 20: Loss - 7139.097600996494
Epoch 21: Loss - 7141.646205067635
Epoch 22: Loss - 7142.809313058853
Epoch 23: Loss - 7140.812252104282
Epoch 24: Loss - 7143.124248385429
Epoch 25: Loss - 7142.831678509712


In [145]:
loss = torch.sqrt(loss_function(test_pred, test_y))
loss.item()

1.0073580741882324